**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Aug  2 10:50:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 15573686464772836946600196822111.jpg
 15573686722502623506381280557123.jpg
 2019-05-09-07-52-17-596.jpg
'acc details ticket.gdoc'
'accommodation n registration robocon (1).docx'
'accommodation n registration robocon (1).gdoc'
'accommodation n registration robocon (2).gdoc'
'accommodation n registration robocon.docx'
'accommodation n registration robocon.gdoc'
'advice letter_ neelesh_nmims.gdoc'
 application.mohit.docx
 application.sandeep.docx
'A Quick Guide

**1) Clone the Darknet**



In [2]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 14163 (delta 5), reused 12 (delta 5), pack-reused 14148
Receiving objects: 100% (14163/14163), 12.77 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (9622/9622), done.


**2) Compile Darknet using Nvidia GPU**


In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [4]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [5]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
#!mkdir "/mydrive/yolov3"

In [6]:
!echo "number_plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/Yolo_V3_for_OpenCV/car_images_number_plate' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-01 11:24:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.47MB/s    in 1m 55s  

2020-08-01 11:26:12 (1.34 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [7]:
!unzip /mydrive/Yolo_V3_for_OpenCV/car_images_number_plate/car_images.zip -d data/obj

Archive:  /mydrive/Yolo_V3_for_OpenCV/car_images_number_plate/car_images.zip
  inflating: data/obj/cfs296.txt     
  inflating: data/obj/cfs297.jpg     
  inflating: data/obj/cfs297.txt     
  inflating: data/obj/cfs298.jpg     
  inflating: data/obj/cfs298.txt     
  inflating: data/obj/cfs299.jpeg    
  inflating: data/obj/cfs299.txt     
  inflating: data/obj/cfs300.jpeg    
  inflating: data/obj/cfs300.txt     
  inflating: data/obj/cfs301.jpeg    
  inflating: data/obj/cfs301.txt     
  inflating: data/obj/cfs302.jpeg    
  inflating: data/obj/cfs302.txt     
  inflating: data/obj/cfs303.jpeg    
  inflating: data/obj/cfs303.txt     
  inflating: data/obj/cfs304.jpeg    
  inflating: data/obj/cfs304.txt     
  inflating: data/obj/cfs305.jpeg    
  inflating: data/obj/cfs305.txt     
  inflating: data/obj/cfs306.jpeg    
  inflating: data/obj/cfs306.txt     
  inflating: data/obj/cfs307.jpeg    
  inflating: data/obj/cfs307.txt     
  inflating: data/obj/cfs308.jpeg    
  inflating

In [8]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.

['15', '0.100625', '0.619167', '0.066250', '0.121667']
77 data/obj/cfs273.txt
0 0.100625 0.619167 0.066250 0.121667
15 0.513750 0.638333 0.142500 0.043333

['15', '0.513750', '0.638333', '0.142500', '0.043333']
78 data/obj/cf93.txt
0 0.513750 0.638333 0.142500 0.043333
15 0.290000 0.528333 0.100000 0.060000

['15', '0.290000', '0.528333', '0.100000', '0.060000']
79 data/obj/cbs263.txt
0 0.290000 0.528333 0.100000 0.060000
15 0.518750 0.718333 0.125000 0.043333

['15', '0.518750', '0.718333', '0.125000', '0.043333']
80 data/obj/cf106.txt
0 0.518750 0.718333 0.125000 0.043333
15 0.356250 0.590833 0.167500 0.118333

['15', '0.356250', '0.590833', '0.167500', '0.118333']
81 data/obj/cfs90.txt
0 0.356250 0.590833 0.167500 0.118333
15 0.498750 0.505000 0.187500 0.056667

['15', '0.498750', '0.505000', '0.187500', '0.056667']
82 data/obj/cb141.txt
0 0.498750 0.505000 0.187500 0.056667
15 0.482500 0.650000 0.202500 0.070000

['15', '0.482500'

In [9]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/cf196.jpg', 'data/obj/cbs161.jpg', 'data/obj/cfs296.jpg', 'data/obj/cf186.jpg', 'data/obj/cbs239.jpg', 'data/obj/cbs141.jpg', 'data/obj/cb147.jpg', 'data/obj/cb70.jpg', 'data/obj/cb60.jpg', 'data/obj/cbs96.jpg', 'data/obj/cf54.jpg', 'data/obj/cbs93.jpg', 'data/obj/cb121.jpg', 'data/obj/cf184.jpg', 'data/obj/cfs71.jpg', 'data/obj/cf136.jpg', 'data/obj/cbs57.jpg', 'data/obj/cbs122.jpg', 'data/obj/cbs43.jpg', 'data/obj/cb206.jpg', 'data/obj/cbs140.jpg', 'data/obj/cb141.jpg', 'data/obj/cfs206.jpg', 'data/obj/cbs155.jpg', 'data/obj/cf141.jpg', 'data/obj/cb54.jpg', 'data/obj/cfs58.jpg', 'data/obj/cb64.jpg', 'data/obj/cb33.jpg', 'data/obj/cb138.jpg', 'data/obj/cbs216.jpg', 'data/obj/cbs48.jpg', 'data/obj/cb14.jpg', 'data/obj/cbs145.jpg', 'data/obj/cfs179.jpg', 'data/obj/cfs121.jpg', 'data/obj/cb111.jpg', 'data/obj/cb69.jpg', 'data/obj/cfs162.jpg', 'data/obj/cb109.jpg', 'data/obj/cfs297.jpg', 'data/obj/cbs41.jpg', 'data/obj/cbs34.jpg', 'data/obj/cfs201.jpg', 'data/obj/cbs163.jpg', '

In [10]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [11]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/Yolo_V3_for_OpenCV/car_images_number_plate/yolov3_training_last.weights -dont_show -map

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.848358, GIOU: 0.847475), Class: 0.998910, Obj: 0.998153, No Obj: 0.000288, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.000080, iou_loss = 0.045271, total_loss = 0.045351 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000011, iou_loss = 0.000000, total_loss = 0.000011 
 total_bbox = 84808, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000319, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.832685, GIOU: 0.830725), Class: 0.998734, Obj: 0.788548, No Obj: 